<a href="https://colab.research.google.com/github/pandeyAnush/facebook/blob/master/heart_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  # <--- RandomForestClassifier is imported here
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE  # Install imblearn using `pip install imblearn`
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = '/content/new_heart.csv'  # Update this path with the actual file path
data = pd.read_csv(file_path)

# Declare X and y as global and assign them
global X, y
X = data.drop(columns='output')
y = data['output']

# Function to show the correlation matrix as a heatmap
def show_correlation_matrix(data):
    correlation_matrix = data.corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Matrix Heatmap')
    plt.show()

# Function to train the model and show test accuracy, MSE, and R2
def train_and_show_accuracy(data):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Oversample using SMOTE
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Initialize the Random Forest classifier with hyperparameters
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)  # <--- RandomForestClassifier is used here

    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf_model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'\nTest Accuracy: {accuracy}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'R-squared (R2): {r2}')

    return rf_model, scaler

# Function to predict heart attack for a new patient
def predict_heart_attack(model, scaler, new_data):
    new_data_df = pd.DataFrame([new_data], columns=X.columns)
    new_data_scaled = scaler.transform(new_data_df)
    prediction = model.predict(new_data_scaled)
    # Remove the following line to remove the percentage calculation
    # probability = model.predict_proba(new_data_scaled)[0][1] * 100  # Get probability of class 1 (heart attack)
    if prediction == 1:
        print(f'Prediction for new patient:Less chance of heart attack')
    else:
        print(f'Prediction for new patient: More chance of heart attack')

# Prompt user for input data
def get_user_input():
    print("Please enter the following details:")
    age = int(input("Age: "))
    sex = int(input("Sex (1 = male, 0 = female): "))
    cp = int(input("Chest Pain Type (0-3): "))
    bp = int(input("Resting Blood Pressure (in mm Hg): "))
    chol = int(input("Serum Cholesterol (in mg/dl): "))
    fbs = int(input("Fasting Blood Sugar > 120 mg/dl (1 = true, 0 = false): "))
    restecg = int(input("Resting ECG results (0-2): "))
    thalachh = int(input("Maximum Heart Rate Achieved: "))
    exng = int(input("Exercise Induced Angina (1 = yes, 0 = no): "))
    oldpeak = float(input("ST Depression Induced by Exercise Relative to Rest: "))
    slp = int(input("Slope of the Peak Exercise ST Segment (0-2): "))
    caa = int(input("Number of Major Vessels (0-3): "))
    thall = int(input("Thallium Stress Test Result (0-3): "))

    return [age, sex, cp, bp, chol, fbs, restecg, thalachh, exng, oldpeak, slp, caa, thall]

# Function to review the data
def review_data():
  while True:
    print("\nData Review Options:")
    print("1. View data information (like .info())")
    print("2. View first 5 rows (like .head())")
    print("3. View last 5 rows (like .tail())")
    print("4. View summary statistics (like .describe())")
    print("5. Calculate Mean Values for Columns")
    print("6. Back to Main Menu")

    choice = input("Enter your choice: ")

    if choice == '1':
      print(data.info())
    elif choice == '2':
      print(data.head())
    elif choice == '3':
      print(data.tail())
    elif choice == '4':
      print("\nSummary Statistics:")
      print(data.describe())
      print("\nKey Statistics:")
      print("  - count: Number of non-missing values in each column")
      print("  - mean: Average value of each column")
      print("  - std: Standard deviation (spread of data)")
      print("  - min: Minimum value in each column")
      print("  - 25%, 50%, 75%: Percentiles (how data is distributed)")
      print("  - max: Maximum value in each column")
    elif choice == '5':
      print("\nMean Values for Columns:")
      for col in data.columns:
          if data[col].dtype == 'int64' or data[col].dtype == 'float64':
              mean_val = data[col].mean()
              print(f"Mean of {col}: {mean_val:.2f}")
    elif choice == '6':
      break
    else:
      print("Invalid choice. Please try again.")

# Main menu system
def main_menu():
    while True:
        print("\nPlease choose an option:")
        print("1. Show Correlation Matrix")
        print("2. Show Test Accuracy, MSE, and R2")
        print("3. Predict Heart Attack")
        print("4. Review Data")
        print("5. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            show_correlation_matrix(data)
        elif choice == '2':
            global rf_model, scaler
            rf_model, scaler = train_and_show_accuracy(data)
        elif choice == '3':
            if 'rf_model' in globals() and 'scaler' in globals():
                new_patient_data = get_user_input()
                predict_heart_attack(rf_model, scaler, new_patient_data)
            else:
                print("Please train the model first by choosing option 2.")
        elif choice == '4':
            review_data()
        elif choice == '5':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the main menu
if __name__ == "__main__":
    main_menu()